In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install vecstack


  Created wheel for vecstack: filename=vecstack-0.4.0-cp37-none-any.whl size=19877 sha256=640622f37131c46e93622687b40de33588e8727badb72d04f44027baa90833bb
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [ ]:
pip install catboost

     |████████████████████████████████| 67.3MB 56kB/s 


# **Importing Required Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LassoCV
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import PolynomialFeatures
from vecstack import stacking
from sklearn.svm import SVR
import statsmodels.api as sm
import catboost as ctb
from sklearn.ensemble import AdaBoostRegressor
from tabulate import tabulate
sns.set()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **Gathering Data**

In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/BSc Project/insurance.csv')

In [ ]:
dataset.shape

(1338, 7)

In [ ]:
dataset.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
dataset.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [ ]:
dataset.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [ ]:
dataset.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

# **Preprocessing**

**Adding BMI Groups**

In [ ]:
conditions = [
    (dataset['bmi'] < 18.5) ,
    (dataset['bmi'] <= 24.9) & (dataset['bmi'] >= 18.5),
    (dataset['bmi'] <= 29.9) & (dataset['bmi'] > 24.9),
    (dataset['bmi'] > 29.9),
]




choices = ['underweight', 'normal', 'overweight', 'obese']


dataset["bmiGps"] = np.select(condlist=conditions, choicelist=choices)
# dataset.drop(["bmi"], axis=1, inplace=True)
dataset.head()

,age,sex,bmi,children,smoker,region,charges,bmiGps
0,19,female,27.900,0,yes,southwest,16884.92400,overweight
1,18,male,33.770,1,no,southeast,1725.55230,obese
2,28,male,33.000,3,no,southeast,4449.46200,obese
3,33,male,22.705,0,no,northwest,21984.47061,normal
4,32,male,28.880,0,no,northwest,3866.85520,overweight


**Adding Age Groups**

In [ ]:
ageconditions = [
    (dataset['age'] <= 35) & (dataset['age'] >= 18),
    (dataset['age'] <= 55) & (dataset['age'] > 35),
    (dataset['age'] > 55),
]




agechoices = ['young-adults', 'middle-aged-adults', 'elder-adults']


dataset["ageGps"] = np.select(condlist=ageconditions, choicelist=agechoices)
# dataset.drop(["bmi"], axis=1, inplace=True)
dataset.head()

,age,sex,bmi,children,smoker,region,charges,bmiGps,ageGps
0,19,female,27.900,0,yes,southwest,16884.92400,overweight,young-adults
1,18,male,33.770,1,no,southeast,1725.55230,obese,young-adults
2,28,male,33.000,3,no,southeast,4449.46200,obese,young-adults
3,33,male,22.705,0,no,northwest,21984.47061,normal,young-adults
4,32,male,28.880,0,no,northwest,3866.85520,overweight,young-adults


**Using Label Encoder for Categorical Features**

In [ ]:
#sex
le = LabelEncoder()
leRegion = LabelEncoder()
le.fit(dataset.sex.drop_duplicates())
dataset.sex = le.transform(dataset.sex)
# smoker
le.fit(dataset.smoker.drop_duplicates())
dataset.smoker = le.transform(dataset.smoker)
#region
leRegion.fit(dataset.region.drop_duplicates())
dataset.region = leRegion.transform(dataset.region)
#BMI Groups
le.fit(dataset.bmiGps.drop_duplicates())
dataset.bmiGps = le.transform(dataset.bmiGps)
#age Groups
le.fit(dataset.ageGps.drop_duplicates())
dataset.ageGps = le.transform(dataset.ageGps)
dataset.head()


,age,sex,bmi,children,smoker,region,charges,bmiGps,ageGps
0,19,0,27.900,0,1,3,16884.92400,2,2
1,18,1,33.770,1,0,2,1725.55230,1,2
2,28,1,33.000,3,0,2,4449.46200,1,2
3,33,1,22.705,0,0,1,21984.47061,0,2
4,32,1,28.880,0,0,1,3866.85520,2,2


**Normalizing Data**

In [ ]:
scaler = preprocessing.MinMaxScaler()
names = dataset.columns
d = scaler.fit_transform(dataset)
scaled_df = pd.DataFrame(d, columns=names)
scaled_df.head()

,age,sex,bmi,children,smoker,region,charges,bmiGps,ageGps
0,0.021739,0.0,0.321227,0.0,1.0,1.000000,0.251611,0.666667,1.0
1,0.000000,1.0,0.479150,0.2,0.0,0.666667,0.009636,0.333333,1.0
2,0.217391,1.0,0.458434,0.6,0.0,0.666667,0.053115,0.333333,1.0
3,0.326087,1.0,0.181464,0.0,0.0,0.333333,0.333010,0.000000,1.0
4,0.304348,1.0,0.347592,0.0,0.0,0.333333,0.043816,0.666667,1.0


# **Splitting Data into Train and Test Sets**

In [ ]:
X = scaled_df.drop(["charges"], axis=1)

y = scaled_df.charges.values

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1070, 8)
Training Labels Shape: (1070,)
Testing Features Shape: (268, 8)
Testing Labels Shape: (268,)


# **Linear Regression Using Statsmodels Library**


[https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9](https://)



In [ ]:
# Add an intercept (beta_0) to our model(Like scikit)
train_features_stats = sm.add_constant(train_features)
test_features_stats = sm.add_constant(test_features)
olsScores=[]
statmodel = sm.OLS(train_labels, train_features_stats).fit()
statmodel.summary()
rtwoTrain = r2_score(train_labels, statmodel.predict(train_features_stats))
olsScores.append(rtwoTrain)
mseTrain = mean_squared_error(statmodel.predict(train_features_stats),train_labels)
olsScores.append(mseTrain)
rtwo = r2_score(test_labels, statmodel.predict(test_features_stats))
olsScores.append(rtwo)
mse = mean_squared_error(statmodel.predict(test_features_stats),test_labels)
olsScores.append(mse)
print('>OLS =>  r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (olsScores[0], olsScores[1], olsScores[2], olsScores[3]))

>OLS =>  r2_score_Train: 0.73720 , MSE_Train: 0.00960 , r2_score_Test: 0.80046 , MSE_Test: 0.00809


In [ ]:
statmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     372.0
Date:                Mon, 24 May 2021   Prob (F-statistic):          1.30e-301
Time:                        14:16:52   Log-Likelihood:                 967.37
No. Observations:                1070   AIC:                            -1917.
Df Residuals:                    1061   BIC:                            -1872.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0472      0.026     -1.782      0.075      -0.099       0.005
age            0.1921      0.026      7.495      0.000       0.142       0.242
sex           -0.0004      0.006     -0.060      0.952      -0.012       0.011
bmi            0.1950      0.018     10.582      0.000       0.159       0.231
children       0.0346      0.013      2.738      0.006       0.010       0.059
smoker         0.3760      0.007     50.247      0.000       0.361       0.391
region        -0.0139      0.008     -1.682      0.093      -0.030       0.002
bmiGps        -0.0154      0.013     -1.195      0.232      -0.041       0.010
ageGps         0.0052      0.022      0.236      0.813      -0.038       0.048
==============================================================================
Omnibus:                      253.812   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              609.504
Skew:                           1.266   Prob(JB):                    4.45e-133
Kurtosis:                       5.694   Cond. No.                         22.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**PLS**

In [ ]:
from sklearn.cross_decomposition import PLSRegression
plsscores=[]
pls = PLSRegression(n_components=1)
pls.fit(train_features, train_labels)
rtwoTrain = r2_score(train_labels, pls.predict(train_features))
plsscores.append(rtwoTrain)
mseTrain = mean_squared_error(pls.predict(train_features),train_labels)
plsscores.append(mseTrain)
rtwo = r2_score(test_labels, pls.predict(test_features))
plsscores.append(rtwo)
mse = mean_squared_error(pls.predict(test_features),test_labels)
plsscores.append(mse)
print('>non-linear Stacking =>  r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (plsscores[0], plsscores[1], plsscores[2], plsscores[3]))

>PLS =>  r2_score_Train: 0.69117 , MSE_Train: 0.01128 , r2_score_Test: 0.76546 , MSE_Test: 0.00951


# **Defining Models and Evaluation Methods**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
level0 = list()
nonlScores=[]
level0.append(('rf', RandomForestRegressor(max_depth= 5, n_estimators=1000)))
level0.append(('linear',LinearRegression()))
level0.append(('knn', KNeighborsRegressor()))
level0.append(('catboost', ctb.CatBoostRegressor(verbose=False, depth = 4, iterations=200, l2_leaf_reg= 0.2, learning_rate= 0.03)))
# define meta learner model
level1 = KNeighborsRegressor()
# define the stacking ensemble
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
crossRes = cross_val_score(model, train_features, train_labels, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
nonlScores.append(mean(crossRes))
model.fit(train_features, train_labels)
rtwo_Train = r2_score(train_labels, model.predict(train_features))
nonlScores.append(rtwo_Train)
mse_Train = mean_squared_error(model.predict(train_features),train_labels)
nonlScores.append(mse_Train)
rtwo = r2_score(test_labels, model.predict(test_features))
nonlScores.append(rtwo)
mse = mean_squared_error(model.predict(test_features),test_labels)
nonlScores.append(mse)
print('>non-linear Stacking => cross_val_score: %.5f , r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (nonlScores[0], nonlScores[1], nonlScores[2] , nonlScores[3], nonlScores[4],))

>non-linear Stacking => cross_val_score: 0.80536 , r2_score_Train: 0.84576 , MSE_Train: 0.00563 , r2_score_Test: 0.86839 , MSE_Test: 0.00534


In [ ]:
def get_stacking():
  level0 = list()
  level0.append(('rf', RandomForestRegressor(max_depth= 5, n_estimators=1000)))
  level0.append(('linear',LinearRegression()))
  level0.append(('knn', KNeighborsRegressor()))
  level0.append(('catboost', ctb.CatBoostRegressor(verbose=False, depth = 4, iterations=200, l2_leaf_reg= 0.2, learning_rate= 0.03)))
	# define meta learner model
  level1 = LinearRegression()
	# define the stacking ensemble
  model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
  return model

In [ ]:

def evaluate_model(model, train_features, train_labels, test_features, test_labels):
  scores =[]
  cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
  crossRes = cross_val_score(model, train_features, train_labels, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
  scores.append(mean(crossRes))
  model.fit(train_features, train_labels)
  rtwoTrain = r2_score(train_labels, model.predict(train_features))
  scores.append(rtwoTrain)
  mseTrain = mean_squared_error(model.predict(train_features),train_labels)
  scores.append(mseTrain)
  rtwo = r2_score(test_labels, model.predict(test_features))
  scores.append(rtwo)
  mse = mean_squared_error(model.predict(test_features),test_labels)
  scores.append(mse)
  return scores

In [ ]:
def get_models():
  models = dict()
  models['rf'] =RandomForestRegressor(n_estimators = 100, random_state = 1,n_jobs = -1)
  models['rf-tuned'] = RandomForestRegressor(max_depth= 5, n_estimators=1000)
  models['linear'] = LinearRegression()
  models['ridge'] = Ridge(alpha=0.5)
  models['lasso'] = LassoCV()
  models['gbr'] = GradientBoostingRegressor()
  models['gbr-tuned'] = GradientBoostingRegressor(max_depth= 2,  n_estimators=100)
  models['xgb'] = XGBRegressor()
  models['xgb-tuned'] = XGBRegressor(colsample_bytree=1,learning_rate=0.01,max_depth=3,n_estimators=500)
  models['knn'] = KNeighborsRegressor()
  models['svr'] = SVR(kernel = 'rbf')
  models['catboost'] = ctb.CatBoostRegressor(verbose=False)
  models['catboost-tuned'] = ctb.CatBoostRegressor(verbose=False, depth = 4, iterations=200, l2_leaf_reg= 0.2, learning_rate= 0.03)
  models['adaboost'] =AdaBoostRegressor(n_estimators=1000)
  models['adaboost-tuned'] =AdaBoostRegressor(n_estimators=100, learning_rate=0.001)
  models['stacking'] = get_stacking()
  return models

In [ ]:
models = get_models()
table=[]
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
  scores = evaluate_model(model, train_features, train_labels, test_features, test_labels)
  results.append(scores)
  names.append(name)
  table.append([name, scores[0], scores[1], scores[2], scores[3], scores[4]] )
  print('>%s => cross_val_score: %.5f , r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (name, scores[0], scores[1], scores[2], scores[3], scores[4]))
# plot model performance for comparison
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot.show()


print(tabulate(table, headers=["Model","Cross_Val_Score", "R2_Score_Train","MSE_Train","R2_Score_Test","MSE_Test"]))

>rf => cross_val_score: 0.80755 , r2_score_Train: 0.97437 , MSE_Train: 0.00094 , r2_score_Test: 0.87541 , MSE_Test: 0.00505
>rf-tuned => cross_val_score: 0.83257 , r2_score_Train: 0.88305 , MSE_Train: 0.00427 , r2_score_Test: 0.89548 , MSE_Test: 0.00424
>linear => cross_val_score: 0.71901 , r2_score_Train: 0.73720 , MSE_Train: 0.00960 , r2_score_Test: 0.80046 , MSE_Test: 0.00809
>ridge => cross_val_score: 0.71917 , r2_score_Train: 0.73717 , MSE_Train: 0.00960 , r2_score_Test: 0.80037 , MSE_Test: 0.00809
>lasso => cross_val_score: 0.71986 , r2_score_Train: 0.73711 , MSE_Train: 0.00960 , r2_score_Test: 0.79989 , MSE_Test: 0.00811
>gbr => cross_val_score: 0.83301 , r2_score_Train: 0.89847 , MSE_Train: 0.00371 , r2_score_Test: 0.89799 , MSE_Test: 0.00414
>gbr-tuned => cross_val_score: 0.83664 , r2_score_Train: 0.87220 , MSE_Train: 0.00467 , r2_score_Test: 0.90085 , MSE_Test: 0.00402
[08:07:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of

In [ ]:
models = get_models()
table=[]
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
  scores = evaluate_model(model, train_features, train_labels, test_features, test_labels)
  results.append(scores)
  names.append(name)
  table.append([name, scores[0], scores[1], scores[2], scores[3], scores[4]] )
  print('>%s => cross_val_score: %.5f , r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (name, scores[0], scores[1], scores[2], scores[3], scores[4]))
# plot model performance for comparison
# pyplot.boxplot(results, labels=names, showmeans=True)
# pyplot.show()


print(tabulate(table, headers=["Model","Cross_Val_Score", "R2_Score_Train","MSE_Train","R2_Score_Test","MSE_Test"]))

>rf => cross_val_score: 0.80755 , r2_score_Train: 0.97437 , MSE_Train: 0.00094 , r2_score_Test: 0.87541 , MSE_Test: 0.00505
>rf-tuned => cross_val_score: 0.83249 , r2_score_Train: 0.88306 , MSE_Train: 0.00427 , r2_score_Test: 0.89523 , MSE_Test: 0.00425
>linear => cross_val_score: 0.71901 , r2_score_Train: 0.73720 , MSE_Train: 0.00960 , r2_score_Test: 0.80046 , MSE_Test: 0.00809
>ridge => cross_val_score: 0.71917 , r2_score_Train: 0.73717 , MSE_Train: 0.00960 , r2_score_Test: 0.80037 , MSE_Test: 0.00809
>lasso => cross_val_score: 0.71986 , r2_score_Train: 0.73711 , MSE_Train: 0.00960 , r2_score_Test: 0.79989 , MSE_Test: 0.00811
>gbr => cross_val_score: 0.83288 , r2_score_Train: 0.89847 , MSE_Train: 0.00371 , r2_score_Test: 0.89794 , MSE_Test: 0.00414
>gbr-tuned => cross_val_score: 0.83648 , r2_score_Train: 0.87220 , MSE_Train: 0.00467 , r2_score_Test: 0.90085 , MSE_Test: 0.00402
[15:23:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of

**Defining Neural Network**

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
scoresNN =[]
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 128, activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 64, activation = 'relu'))

classifier.add(Dense(units = 32, activation = 'relu'))

classifier.add(Dense(units = 4, activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, activation = 'linear'))

# Compiling the ANN
classifier.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
classifier.fit(x = train_features, y = train_labels, batch_size = 16, epochs = 100,validation_split=0.33)
rtwoTrain = r2_score(train_labels, classifier.predict(train_features))
scoresNN.append(rtwoTrain)
mseTrain = mean_squared_error(classifier.predict(train_features),train_labels)
scoresNN.append(mseTrain)
rtwo = r2_score(test_labels, classifier.predict(test_features))
scoresNN.append(rtwo)
mse = mean_squared_error(classifier.predict(test_features),test_labels)
scoresNN.append(mse)
print('>Neural Network =>  r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (scoresNN[0], scoresNN[1], scoresNN[2], scoresNN[3]))

Epoch 1/100
45/45 [==============================] - 1s 13ms/step - loss: 0.0768 - mean_absolute_error: 0.0768 - val_loss: 0.0597 - val_mean_absolute_error: 0.0597
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 0.0546 - mean_absolute_error: 0.0546 - val_loss: 0.0544 - val_mean_absolute_error: 0.0544
Epoch 3/100
45/45 [==============================] - 0s 3ms/step - loss: 0.0399 - mean_absolute_error: 0.0399 - val_loss: 0.0488 - val_mean_absolute_error: 0.0488
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.0365 - mean_absolute_error: 0.0365 - val_loss: 0.0473 - val_mean_absolute_error: 0.0473
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 0.0322 - mean_absolute_error: 0.0322 - val_loss: 0.0425 - val_mean_absolute_error: 0.0425
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 0.0329 - mean_absolute_error: 0.0329 - val_loss: 0.0384 - val_mean_absolute_error: 0.0384
Epoch 7/100
45/45 [==

**Defining Ploynomial Model with the optimal degree (Based on Experiments)**

In [ ]:
# poly-degree:2
poly_features = PolynomialFeatures(degree=2)
X_poly = poly_features.fit_transform(train_features)
poly = LinearRegression()
polyScore =[]
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
crossRes = cross_val_score(poly, train_features, train_labels, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
polyScore.append(mean(crossRes))
poly.fit(X_poly, train_labels)
X_poly_test = poly_features.fit_transform(test_features)
rtwo_Train = r2_score(train_labels, poly.predict(X_poly))
polyScore.append(rtwo_Train)
mse_Train = mean_squared_error(poly.predict(X_poly),train_labels)
polyScore.append(mse_Train)
rtwo = r2_score(test_labels, poly.predict(X_poly_test))
polyScore.append(rtwo)
mse = mean_squared_error(poly.predict(X_poly_test),test_labels)
polyScore.append(mse)
print('>polynomial-degree2 => cross_val_score: %.5f , r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (polyScore[0], polyScore[1], polyScore[2] , polyScore[3], polyScore[4],))

>polynomial-degree2 => cross_val_score: 0.71901 , r2_score_Train: 0.84469 , MSE_Train: 0.00567 , r2_score_Test: 0.88254 , MSE_Test: 0.00476


**Testing Another Method for creating a Stack Model**

In [ ]:
  level0 = list()
  level0.append(RandomForestRegressor(max_depth= 5, max_features=7, n_estimators=1000))
  level0.append(Ridge(alpha=0.5))
  level0.append(GradientBoostingRegressor(max_depth= 2,  n_estimators=100))
  level0.append( KNeighborsRegressor())
  level0.append( LassoCV())
  level0.append( XGBRegressor(colsample_bytree=1,learning_rate=0.01,max_depth=3,n_estimators=500))




  S_train, S_test = stacking(level0,
                           train_features, train_labels, test_features,
                           regression=True,

                           mode='oof_pred_bag',

                           needs_proba=False,

                           save_dir=None,

                           metric=r2_score,

                           n_folds=5,

                           stratified=True,

                           shuffle=True,

                           random_state=0,

                           verbose=2)

	# define meta learner model
  model = LinearRegression()
  scoresVecstack =[]
  cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
  crossRes = cross_val_score(model, S_train, train_labels, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
  scoresVecstack.append(mean(crossRes))
  model.fit(S_train, train_labels)
  y_pred = model.predict(S_test)
  rtwo = r2_score(test_labels,y_pred )
  scoresVecstack.append(rtwo)
  mse = mean_squared_error(y_pred, test_labels)
  scoresVecstack.append(mse)
  print(scoresVecstack)


/usr/local/lib/python3.7/dist-packages/vecstack/core.py:461: UserWarning: This is regression task hence classification-specific parameters set to <True> were ignored: <stratified>
  warnings.warn(warn_str, UserWarning)


task:         [regression]
metric:       [r2_score]
mode:         [oof_pred_bag]
n_models:     [6]

model  0:     [RandomForestRegressor]
    fold  0:  [0.84302232]
    fold  1:  [0.83482451]
    fold  2:  [0.84489067]
    fold  3:  [0.87384196]
    fold  4:  [0.81945312]
    ----
    MEAN:     [0.84320652] + [0.01775321]
    FULL:     [0.84430324]

model  1:     [Ridge]
    fold  0:  [0.71082437]
    fold  1:  [0.74700732]
    fold  2:  [0.75439317]
    fold  3:  [0.74611643]
    fold  4:  [0.70969066]
    ----
    MEAN:     [0.73360639] + [0.01928294]
    FULL:     [0.73483876]

model  2:     [GradientBoostingRegressor]
    fold  0:  [0.84718569]
    fold  1:  [0.84451835]
    fold  2:  [0.85075979]
    fold  3:  [0.86492860]
    fold  4:  [0.82484166]
    ----
    MEAN:     [0.84644682] + [0.01289361]
    FULL:     [0.84731489]

model  3:     [KNeighborsRegressor]
    fold  0:  [0.80090653]
    fold  1:  [0.79391628]
    fold  2:  [0.80281190]
    fold  3:  [0.82486181]
    fold  4:

# **Reducing Features to 3 most Important Features**

In [ ]:
X2 = scaled_df.drop(["charges", "sex", "region", "bmiGps","ageGps", "children"], axis=1)
reduced_train_features, reduced_test_features, reduced_train_labels, reduced_test_labels = train_test_split(X2, y, test_size=0.2, random_state=0)
print('Training Features Shape:', reduced_train_features.shape)
print('Training Labels Shape:', reduced_train_labels.shape)
print('Testing Features Shape:', reduced_test_features.shape)
print('Testing Labels Shape:', reduced_test_labels.shape)

Training Features Shape: (1070, 3)
Training Labels Shape: (1070,)
Testing Features Shape: (268, 3)
Testing Labels Shape: (268,)


In [ ]:
Reduced_statmodel = sm.OLS(reduced_train_labels, reduced_train_features).fit()
predictions = Reduced_statmodel.predict(reduced_test_features)
Reduced_statmodel.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.865
Model:                            OLS   Adj. R-squared (uncentered):              0.864
Method:                 Least Squares   F-statistic:                              2276.
Date:                Fri, 07 May 2021   Prob (F-statistic):                        0.00
Time:                        07:23:03   Log-Likelihood:                          947.58
No. Observations:                1070   AIC:                                     -1889.
Df Residuals:                    1067   BIC:                                     -1874.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
age            0.1661      0.009     18.309      0.000       0.148       0.184
bmi            0.1173      0.012      9.835      0.000       0.094       0.141
smoker         0.3688      0.007     49.740      0.000       0.354       0.383
==============================================================================
Omnibus:                      269.458   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              660.358
Skew:                           1.335   Prob(JB):                    4.03e-144
Kurtosis:                       5.772   Cond. No.                         3.25
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
models = get_models()
table=[]
# evaluate the models and store results
results, names = list(), list()
print("< Reducing Features to Features Selected by the Feature Selection Algorithms > \n")
for name, model in models.items():
  scores = evaluate_model(model, reduced_train_features, reduced_train_labels, reduced_test_features, reduced_test_labels)
  results.append(scores)
  names.append(name)
  table.append([name, scores[0], scores[1], scores[2], scores[3], scores[4]] )
  print('>%s => cross_val_score: %.5f , r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (name, scores[0], scores[1], scores[2], scores[3], scores[4]))


print("\n")
print(tabulate(table, headers=["Model","Cross_Val_Score", "R2_Score_Train","MSE_Train","R2_Score_Test","MSE_Test"]))



< Reducing Features to Features Selected by the Feature Selection Algorithms > 

>rf => cross_val_score: 0.78425 , r2_score_Train: 0.96840 , MSE_Train: 0.00115 , r2_score_Test: 0.85685 , MSE_Test: 0.00580
>rf-tuned => cross_val_score: 0.82877 , r2_score_Train: 0.87695 , MSE_Train: 0.00449 , r2_score_Test: 0.88953 , MSE_Test: 0.00448
>linear => cross_val_score: 0.71930 , r2_score_Train: 0.73425 , MSE_Train: 0.00971 , r2_score_Test: 0.79455 , MSE_Test: 0.00833
>ridge => cross_val_score: 0.71940 , r2_score_Train: 0.73424 , MSE_Train: 0.00971 , r2_score_Test: 0.79434 , MSE_Test: 0.00834
>lasso => cross_val_score: 0.71934 , r2_score_Train: 0.73425 , MSE_Train: 0.00971 , r2_score_Test: 0.79446 , MSE_Test: 0.00833
>gbr => cross_val_score: 0.82764 , r2_score_Train: 0.89052 , MSE_Train: 0.00400 , r2_score_Test: 0.89073 , MSE_Test: 0.00443
>gbr-tuned => cross_val_score: 0.83245 , r2_score_Train: 0.86665 , MSE_Train: 0.00487 , r2_score_Test: 0.89602 , MSE_Test: 0.00422
[08:22:53] WARNING: /worksp

In [ ]:
models = get_models()
table=[]
# evaluate the models and store results
results, names = list(), list()
print("< Reducing Features to Features Selected by the Feature Selection Algorithms > \n")
for name, model in models.items():
  scores = evaluate_model(model, reduced_train_features, reduced_train_labels, reduced_test_features, reduced_test_labels)
  results.append(scores)
  names.append(name)
  table.append([name, scores[0], scores[1], scores[2], scores[3], scores[4]] )
  print('>%s => cross_val_score: %.5f , r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (name, scores[0], scores[1], scores[2], scores[3], scores[4]))


print("\n")
print(tabulate(table, headers=["Model","Cross_Val_Score", "R2_Score_Train","MSE_Train","R2_Score_Test","MSE_Test"]))



< Reducing Features to Features Selected by the Feature Selection Algorithms > 

>rf => cross_val_score: 0.78425 , r2_score_Train: 0.96840 , MSE_Train: 0.00115 , r2_score_Test: 0.85685 , MSE_Test: 0.00580
>rf-tuned => cross_val_score: 0.82889 , r2_score_Train: 0.87690 , MSE_Train: 0.00450 , r2_score_Test: 0.89016 , MSE_Test: 0.00445
>linear => cross_val_score: 0.71930 , r2_score_Train: 0.73425 , MSE_Train: 0.00971 , r2_score_Test: 0.79455 , MSE_Test: 0.00833
>ridge => cross_val_score: 0.71940 , r2_score_Train: 0.73424 , MSE_Train: 0.00971 , r2_score_Test: 0.79434 , MSE_Test: 0.00834
>lasso => cross_val_score: 0.71934 , r2_score_Train: 0.73425 , MSE_Train: 0.00971 , r2_score_Test: 0.79446 , MSE_Test: 0.00833
>gbr => cross_val_score: 0.82792 , r2_score_Train: 0.89052 , MSE_Train: 0.00400 , r2_score_Test: 0.89073 , MSE_Test: 0.00443
>gbr-tuned => cross_val_score: 0.83249 , r2_score_Train: 0.86665 , MSE_Train: 0.00487 , r2_score_Test: 0.89602 , MSE_Test: 0.00422
[08:24:19] WARNING: /worksp

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
scoresNN =[]
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 128, activation = 'relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 64, activation = 'relu'))

classifier.add(Dense(units = 32, activation = 'relu'))

classifier.add(Dense(units = 4, activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, activation = 'linear'))

# Compiling the ANN
classifier.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
classifier.fit(x = reduced_train_features, y = reduced_train_labels, batch_size = 16, epochs = 100,validation_split=0.33)
rtwoTrain = r2_score(reduced_train_labels, classifier.predict(reduced_train_features))
scoresNN.append(rtwoTrain)
mseTrain = mean_squared_error(classifier.predict(reduced_train_features),reduced_train_labels)
scoresNN.append(mseTrain)
rtwo = r2_score(reduced_test_labels, classifier.predict(reduced_test_features))
scoresNN.append(rtwo)
mse = mean_squared_error(classifier.predict(reduced_test_features),reduced_test_labels)
scoresNN.append(mse)
print('>Neural Network =>  r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (scoresNN[0], scoresNN[1], scoresNN[2], scoresNN[3]))

Epoch 1/100
45/45 [==============================] - 1s 6ms/step - loss: 0.0954 - mean_absolute_error: 0.0954 - val_loss: 0.0570 - val_mean_absolute_error: 0.0570
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 0.0450 - mean_absolute_error: 0.0450 - val_loss: 0.0472 - val_mean_absolute_error: 0.0472
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 0.0417 - mean_absolute_error: 0.0417 - val_loss: 0.0445 - val_mean_absolute_error: 0.0445
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 0.0352 - mean_absolute_error: 0.0352 - val_loss: 0.0449 - val_mean_absolute_error: 0.0449
Epoch 5/100
45/45 [==============================] - 0s 3ms/step - loss: 0.0353 - mean_absolute_error: 0.0353 - val_loss: 0.0448 - val_mean_absolute_error: 0.0448
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 0.0343 - mean_absolute_error: 0.0343 - val_loss: 0.0461 - val_mean_absolute_error: 0.0461
Epoch 7/100
45/45 [===

In [ ]:
# poly-degree:2
poly_features = PolynomialFeatures(degree=2)
X_poly = poly_features.fit_transform(reduced_train_features)
poly = LinearRegression()
polyScore =[]
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
crossRes = cross_val_score(poly, reduced_train_features, reduced_train_labels, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
polyScore.append(mean(crossRes))
poly.fit(X_poly, reduced_train_labels)
X_poly_test = poly_features.fit_transform(reduced_test_features)
rtwo_Train = r2_score(reduced_train_labels, poly.predict(X_poly))
polyScore.append(rtwo_Train)
mse_Train = mean_squared_error(poly.predict(X_poly),reduced_train_labels)
polyScore.append(mse_Train)
rtwo = r2_score(reduced_test_labels, poly.predict(X_poly_test))
polyScore.append(rtwo)
mse = mean_squared_error(poly.predict(X_poly_test),reduced_test_labels)
polyScore.append(mse)
print('>polynomial-degree2 => cross_val_score: %.5f , r2_score_Train: %.5f , MSE_Train: %.5f , r2_score_Test: %.5f , MSE_Test: %.5f' % (polyScore[0], polyScore[1], polyScore[2] , polyScore[3], polyScore[4],))

>polynomial-degree2 => cross_val_score: 0.71930 , r2_score_Train: 0.82799 , MSE_Train: 0.00628 , r2_score_Test: 0.87733 , MSE_Test: 0.00497


**Conclusion:**

> The impact of Reducing Features on the results is not noticible

However,

>It reduces the computation cost

So it is a wise choice !